<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Рекуррентная нейронная сеть и </b> <span style="font-weight:bold; color:green">Keras</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<p>Подключение стилей оформления</p>

In [ ]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

In [ ]:
import inspect
import time

In [ ]:
import numpy as np
import pickle

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from lib import rnn_lang_modeling_reader as reader

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Вариант 1.</b> Из командной строки</p>

In [ ]:
!wget wget -P data/rnn-lang-modeling/ http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

In [ ]:
!tar xvf data/rnn-lang-modeling/simple-examples.tgz -C data/rnn-lang-modeling/

<p><b>Вариант 2.</b> Средствами Python</p>

In [ ]:
import urllib.request
import shutil
import os
import tarfile

In [ ]:
url = "http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz"

filename = "data/rnn-lang-modeling/rnn-simple.tgz"

os.makedirs(os.path.dirname(filename), exist_ok=True)

# Загрузка архива
with urllib.request.urlopen(url) as response:
    with open(filename, 'wb') as output:
        shutil.copyfileobj(response, output)

# Распаковка
with tarfile.open(filename) as tar:
    tar.extractall(path="data/rnn-lang-modeling/")

Директория с исходными данными и для записи логов и модели

In [ ]:
data_path = "data/rnn-lang-modeling/simple-examples/data"
save_path = "log/rnn-lang-modeling/log"

Загрузка данных и преобразование в вектор индексов слов

In [ ]:
train_data, valid_data, test_data, vocabulary = reader.ptb_raw_data(data_path)

In [ ]:
train_data[:10]

Количество слов (токенов) в обучающем подмножестве

In [ ]:
len(train_data)

Индекс первого слова в тестовом подмножестве

In [ ]:
test_data[0]

Словарь преобразования слов в индексы

In [ ]:
vocabulary

In [ ]:
len(vocabulary)

Обратное пребразование индекса первого слова тестового подмножества

In [ ]:
for key in vocabulary:
    if vocabulary[key] == test_data[0]:
        print(key)

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Этапы построения сети</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

#### Параметры модели

Структура сети

In [ ]:
num_steps = 24 # количество развертки LSTM
hidden_size = 200 # количество LSTM единиц
vocab_size = 10000 # размер словаря

Параметры инициализации весов и параметры обучения

In [ ]:
learning_rate = 1.0 # коэффициент скорости обучения
max_grad_norm = 5 # предельно допустимая норма градиента
max_max_epoch = 4 # количество эпох
batch_size = 20 # размер batch
num_steps_test = 1 # количество развертки LSTM при тестировании
batch_size_test = 1 # размер batch при тестировании

In [ ]:
epoch_size = ((len(train_data) // batch_size) - 1) // num_steps # размер epoch

#### Построение сети

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(vocab_size, hidden_size, input_length=num_steps, trainable=False))
                        #batch_input_shape=(batch_size, num_steps), trainable=False))

In [ ]:
model.add(LSTM(hidden_size, return_sequences=True))#, bias_initializer='zeros', stateful=True))

In [ ]:
model.add(LSTM(hidden_size, return_sequences=False))#, bias_initializer='zeros', stateful=True))

In [ ]:
model.add(Dense(vocab_size))

In [ ]:
model.add(Activation('softmax'))

In [ ]:
optimizer = SGD(lr=learning_rate, clipnorm=max_grad_norm)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

<b>LSTM архитектура сети</b>

<img src="images/rnn-lang-modeling/lstm_architecture.png" width="541px">

Размерность входных текстовых данных упорядочена следующим образом: (размер batch, количество развертки LSTM, количество LSTM единиц). 

Так для каждого batch и каждого слова в развертке LSTM существует вектор слоя внедрения длиной 200 для представления входного слова. Входные данные подаются в два «сложенных» слоя слотов LSTM. Вывод из этих развернутых слотов остается неизменным (размер batch, количество развертки LSTM, количество LSTM единиц).

Затем выходные данные передаются в полностью связанный слой Dense, на котором применяется функция активации softmax, возвращая массив вероятностных оценок. Оценки сравниваются с данными обучения y для каждой соты, затем выполняется обратное распространение ошибки и градиента. 

Так на каждом временном шаге модель пытается предсказать следующее следующее слово в последовательности.


<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Запуск обучения</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

Функция для обучения модели

In [ ]:
def run_epoch(model, input_data, batch_size, num_steps, verbose=False):
    
    start_time = time.time()
    losses = 0.0
    iters = 0
    
    model.reset_states()
    
    for step, (x, y) in enumerate(reader.ptb_iterator(input_data, batch_size, num_steps)):
        y = to_categorical(y, num_classes=vocab_size)
        loss = model.train_on_batch(x, y)
        losses += loss
        iters += num_steps
        
        if verbose and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                (step * 1.0 / epoch_size, np.exp(losses / iters),
                iters * batch_size / (time.time() - start_time)))
    
    return np.exp(losses / iters)  

Запуск обучения

In [ ]:
for i in range(max_max_epoch):

    print("Epoch: %d" % (i + 1))

    train_perplexity = run_epoch(model, train_data, batch_size, num_steps, verbose=True)
    print("Train Perplexity: %.3f" % train_perplexity)

    valid_perplexity = run_epoch(model, valid_data, batch_size, num_steps)
    print("Valid Perplexity: %.3f" % valid_perplexity + "\n")

test_perplexity = run_epoch(model, test_data, batch_size, num_steps) #batch_size_test & num_steps_test -> 
                                                                     #error embedding shape
print("Test Perplexity: %.3f" % test_perplexity)

Метрика <b>Perplexity</b> (растерянность) отражает распределение вероятностей предсказания объекта p и считается по следующей формуле:

<img src="images/rnn-lang-modeling/perplexity.png" width="231px">

В обработке естественного языка растерянность позволяет оценить языковые модели, подсчитывая обратную вероятность появления каждого последующего слова в сгенерированном тексте на основе распределения вероятностей обучающей выборки и выражается в двойке в положительной степени (чем ближе этот показатель к двум, тем точнее работает модель).

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Генерация текста</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

Преобразование индексов в слова

In [ ]:
id_to_word = {}
for c, i in vocabulary.items():
    id_to_word[i] = c

Входная последовательность

In [ ]:
seed_text = "raising money for"

Размер текста

In [ ]:
text_length = 200

Генерация текста

In [ ]:
feed = seed_text
sentence = [vocabulary[word] for word in seed_text.split(" ")]

for i in range(text_length):
    preds = model.predict(pad_sequences([sentence] * batch_size,
                                       maxlen=num_steps))[0]
    
    preds = np.log(preds) / 1.5
    preds = np.exp(preds) / np.sum(np.exp(preds))
    next_index = np.argmax(np.random.multinomial(1, preds, 1))
    sentence = sentence[1:] + next_index
    
    next_word = id_to_word[next_index]
    feed = feed + " " + next_word

print(feed)

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

https://github.com/tensorflow/models/tree/master/tutorials/rnn